# Roll-your-own density functional theory

In [1]:
import numpy as np
from matplotlib import pyplot
%matplotlib inline

### The hydrogen atom

We first start with the H atom, for simplicity. No xc functional is needed, so we are just solving the electronic structure problem in a box with a potential defined by the point charge of the nucleus in the center of the box, at (0, 0, 0). Our objective is to write the Hamiltonian, which is (without nuclear-nuclear repulsion terms:

$$ \hat{h} = - \frac{\hbar^2}{2m} \nabla^2 - e^2 \frac{Z}{\vec{R} - \vec{r}} $$

We first set up the grid.

xs, ys, and zs are now flattened vectors, the elements of which correspond to the coordinates in 3D space of the original grid. Next, we define the potential.

In [2]:
g = 30  # Number of grid points in each dimension.
g3 = g**3.
p = np.linspace(-5., 5., g)
xs, ys, zs = np.meshgrid(p, p, p)
h = p[1] - p[0]  # Grid spacing.
xs = xs.flatten()
ys = ys.flatten()
zs = zs.flatten()

Define the constants. Initially, we start off with atomic units, so everything is unity. Later, we can add units.

In [3]:
hbar = 1.0
m = 1.0
e = 1.0

In [4]:
distances = np.sqrt( xs**2. + ys**2. + zs**2.)

In [5]:
Vext = -1.0/distances * e**2.

Next, we convert this to a sparse matrix

In [6]:
import scipy
from scipy import sparse
Vext = sparse.spdiags(Vext,0,g3,g3)

Now we create the Laplacian. Start with the identity matrix.

In [7]:
I = sparse.eye(g)

The first part of the Laplacian, $[1, -2, 1]$. comes from the finite difference approximation for the second derivative:
$$ \frac{\partial^2 f \left( x \right) }{\partial x^2} \approx \frac{f_{i+1} - 2f_i + f_{i-1}}{\left( \Delta x \right)^2} $$
The first $L$ is our one-dimensional Laplacian.

In [8]:
_ = np.ones(g)
L = sparse.spdiags( [_, -2.0*_, _],[-1,0,1], g, g) / h**2.

(here we're using Andy's throwaway underscore instead of 'e', because using 'e' is silly.). The next bit of magic is the Kronecker tensor product to form the full 3D Laplacian.

In [9]:
L3 = sparse.kron( sparse.kron(L,I),I) + sparse.kron( sparse.kron(I,L),I) + sparse.kron( sparse.kron(I,I),L)
np.shape(L3)
T = -0.5 * hbar**2. / m * L3 

Form the Hamiltonian

In [10]:
Hamiltonian = T + Vext

In [11]:
from scipy.sparse import linalg
eigenvalues, eigenvectors = sparse.linalg.eigs(Hamiltonian,k=3, which='SR')

In [15]:
print "energy of hydrogen atom in eV: %.3F"%(eigenvalues[0] * 27.21)
print eigenvectors

energy of hydrogen atom in eV: -13.216
[[  9.80265367e-07+0.j  -3.61400830e-06+0.j  -1.39196729e-05+0.j]
 [  1.98930017e-06+0.j  -7.60595997e-06+0.j  -2.75965279e-05+0.j]
 [  3.05244134e-06+0.j  -1.23363478e-05+0.j  -4.07628194e-05+0.j]
 ..., 
 [  3.05244134e-06+0.j   1.23363478e-05+0.j   4.07628194e-05+0.j]
 [  1.98930017e-06+0.j   7.60595997e-06+0.j   2.75965279e-05+0.j]
 [  9.80265367e-07+0.j   3.61400830e-06+0.j   1.39196729e-05+0.j]]


/home/aap/.local/lib/python2.7/site-packages/ipykernel/__main__.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  if __name__ == '__main__':


In [18]:
sum(eigenvectors[1]**2.)

(8.2337629689031268e-10+0j)

In [20]:
eigenvectors[0]

array([  9.80265367e-07+0.j,  -3.61400830e-06+0.j,  -1.39196729e-05+0.j])